In [1]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [2]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [3]:
!pip install -q toolz
!pip install -q wandb

In [4]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
csv = pd.read_csv("ZINC_310k.csv")

In [7]:
smiles = csv['SMILES']

In [8]:
smiles = smiles[:60000]

In [9]:
labels = torch.tensor(csv['QED'][:60000]).float()

In [10]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [11]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

In [12]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [13]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [14]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [15]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [16]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [17]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [18]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [19]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [20]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [21]:
train_loader = SemiMolTreeFolder(
    X_train,
    L_train,
    runner.vocab,
    conf["batch_size"],
    label_pct=0.05,
    num_workers=conf["num_workers"],
)

In [22]:
test_loader = SemiMolTreeFolderTest(
    X_test,
    L_test,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [23]:
batch_size_val = math.floor(len(X_Val) / 10)

val_loader = SemiMolTreeFolderTest(
    X_Val,
    L_Val,
    runner.vocab,
    batch_size_val,
    num_workers=conf["num_workers"],
)

In [24]:
print("Training model...")
runner.train_gen_pred(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 0.000, Beta: 0.000, Loss: 112.09, KL: 466.50, MAE: 1.08027, Word Loss: 84.37, Topo Loss: 19.18, Assm Loss: 8.55, Pred Loss: 66.24, Word: 31.94, Topo: 85.23, Assm: 57.43, PNorm: 101.98, GNorm: 50.00
[Train][200] Alpha: 0.000, Beta: 0.000, Loss: 70.98, KL: 376.40, MAE: 0.27236, Word Loss: 51.97, Topo Loss: 11.17, Assm Loss: 7.84, Pred Loss: 4.16, Word: 54.79, Topo: 92.18, Assm: 61.59, PNorm: 106.53, GNorm: 50.00
[Train][300] Alpha: 0.000, Beta: 0.000, Loss: 59.61, KL: 447.63, MAE: 0.23094, Word Loss: 42.58, Topo Loss: 9.60, Assm Loss: 7.43, Pred Loss: 3.22, Word: 63.21, Topo: 93.28, Assm: 63.25, PNorm: 110.28, GNorm: 39.53
[Train][400] Alpha: 0.000, Beta: 0.000, Loss: 51.87, KL: 548.61, MAE: 0.23011, Word Loss: 36.93, Topo Loss: 7.89, Assm Loss: 7.05, Pred Loss: 3.19, Word: 67.73, Topo: 94.44, Assm: 65.72, PNorm: 114.18, GNorm: 50.00
learning rate: 0.000729
[Train][500] Alpha: 0.000, Beta: 0.010, Loss: 48.48, KL: 341.94, MAE: 0.1

KeyboardInterrupt: 

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)